# Gibbs Sampling Inference with LDA

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import lda.datasets as data

## Test Documents

In [2]:
doc = np.array([[2, 3, 7, 2, 1, 5, 6, 8, 1, 2]])

# Real Data

In [3]:
X = data.load_reuters()
vocab = data.load_reuters_vocab()
titles = data.load_reuters_titles()

## Parameters

In [4]:
# Parameters
# per document topic
alpha = 1
# per topic word distribution
beta = 1

In [19]:
# number of stuff.
n_docs = 1
n_topics = 5
n_words =len(vocab)

In [20]:
# topic dist for doc m. Shape = (n_docs, n_topics)
topic_d = np.ones((n_docs, n_topics))
# word dis per topic. Shape = (n_topics, n_words)
word_d = np.ones((n_topics, n_words))
# word assignment per document. Shape = (n_docs, n_words)
word_assign = np.ones((n_docs, n_words))

## Initialisation

In [21]:
def init():
    for i_docs in range (n_docs):
        topic_d[i_docs] = np.random.dirichlet(alpha * np.ones(n_topics))
        word_assign[i_docs] = np.random.randint(n_topics, size=n_words)
    
    for i in range (n_topics):
        word_d[i] = np.random.dirichlet(alpha * np.ones(n_words))

## Gibbs Sampling - So slow

In [22]:
def topic_gibbs(doc, max_iter = 10):
    for i in range(max_iter):
        print(i)
        # Sample from conditional of word assignment
        for i_doc in range(n_docs):
            for i_word in range(n_words):
                # Calculate paramters
                # using prob dist of the topic per doc and conditional topic distribution given words
                param = np.exp(np.log(topic_d[i_doc]) + np.log(word_d[:, doc[i_doc, i_word]]))
                para = param / np.sum(param)
                
                # Redistribute topics w/ new paramters
                topic = np.random.multinomial(1,para)
                word_assign[i_doc, i_word] = np.argmax(topic)
        
        print("Words Assigned!")
        # Sample from conditional of topic distribution per document
        for i_doc in range (n_docs):
            k = np.ones(n_topics)
            for i_topic in range (n_topics):
                k[i_topic] = np.sum(word_assign[i_doc] == i_topic)
            topic_d[i_doc] = np.random.dirichlet(alpha + k)
        
        print("topic_d Updated!")
        # Sample from condition of word distribution per document
        for i_topic in range(n_topics):
            k = np.ones(n_words)
            # Calculate the indicator
            for i_word in range (n_words):
                for i_doc in range (n_docs):
                    for i_wword in range (n_words):
                        k[i_word] += (doc[i_doc , i_wword] == i_word) and (word_assign[i_doc, i_wword] == i_topic)
            
            word_d[i_topic] = np.random.dirichlet(alpha + k)
        print("Iteration Ended!")

In [23]:
init()
topic_gibbs(X)

0
Words Assigned!
topic_d Updated!
Iteration Ended!
1
Words Assigned!
topic_d Updated!
Iteration Ended!
2
Words Assigned!
topic_d Updated!
Iteration Ended!
3
Words Assigned!
topic_d Updated!
Iteration Ended!
4
Words Assigned!
topic_d Updated!
Iteration Ended!
5
Words Assigned!
topic_d Updated!
Iteration Ended!
6
Words Assigned!
topic_d Updated!
Iteration Ended!
7
Words Assigned!
topic_d Updated!
Iteration Ended!
8
Words Assigned!
topic_d Updated!
Iteration Ended!
9
Words Assigned!
topic_d Updated!
Iteration Ended!


In [34]:
n_print = 5
for n_topic in range(n_topics):
    print("topic: ",n_topic)
    for i in range(n_print):
        word_i = word_d[n_topic].argsort()[::-1][i]
        print("vocab: ",vocab[word_i], " prob: ", word_d[n_topic, word_i])

topic:  0
vocab:  van  prob:  0.00126023003762
vocab:  challenges  prob:  0.00107183015909
vocab:  subsidiaries  prob:  0.00107007053527
vocab:  pioneered  prob:  0.00104141360796
vocab:  gays  prob:  0.00100255347129
topic:  1
vocab:  arrested  prob:  0.00129081560686
vocab:  concern  prob:  0.00119666221046
vocab:  receiving  prob:  0.0010377280602
vocab:  threats  prob:  0.0010130547837
vocab:  kohl  prob:  0.00100585488391
topic:  2
vocab:  baptists  prob:  0.00127093322637
vocab:  wind  prob:  0.00114419516328
vocab:  traditional  prob:  0.0010814066262
vocab:  residents  prob:  0.00105761979742
vocab:  message  prob:  0.00104520962987
topic:  3
vocab:  church  prob:  0.324297824692
vocab:  pope  prob:  0.00955767554218
vocab:  years  prob:  0.00151826331138
vocab:  together  prob:  0.000917672155762
vocab:  tourists  prob:  0.000808162570776
topic:  4
vocab:  iliescu  prob:  0.00124255570118
vocab:  industry  prob:  0.00121853144343
vocab:  operation  prob:  0.00117822420558
voca